# NLP2022 - Homework 2


This notebook contains code for a fast processing of data and experiments execution for the second homework of the course Natural Language Processing 2022. It has been completely wrote by Dennis Rotondi 1834864 using the methodologies learned throughout the course.

In [9]:
# imports and deterministic stuff
import os, sys
sys.path.append(os.path.join("..")) #to access hw2 functions
sys.path.append(os.path.join("../..")) #to access model folder

import torch
import numpy as np
import random
import pytorch_lightning as pl
from collections import OrderedDict, Counter

from utils import read_dataset
import wandb
from pytorch_lightning.loggers.wandb import WandbLogger
wandb.require("service")

np.random.seed(0)
random.seed(0)
torch.cuda.manual_seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False
_ = pl.seed_everything(0)

Global seed set to 0


## Dataset Analysis

As for the bonus exercise and hw1, I want to start by looking at the data I have to better understand how to proceed in the pre-processing operations. I've read that there are problems with some (sentence-ground truth) pairs, since we are not allowed to do any change I'll directly discharge them for the training phase if needed.

In [ ]:
en_file = "../../data/EN/train.json"

sentences, labels = read_dataset(en_file)
# I'm just playing with the field of a sentence_id to understand our data samples, I've it also in the homework slides
# but I want to have a reference here
sentence_id = '1996/a/50/18_supp__323:5'
print("## SENTENCE {} ##".format(sentence_id))
for key in sentences[sentence_id]:
    print(key)
    print(sentences[sentence_id][key])
print("## LABEL ##")
for key in labels[sentence_id]:
    print(key)
    print(labels[sentence_id][key])

#let's check and count the different frames and roles
verbatlas_frames = Counter()
predicate_roles = Counter()

for k in labels:
    verbatlas_frames.update(labels[k]['predicates'])
    for idx in labels[k]['roles']:
        predicate_roles.update(labels[k]['roles'][idx])

In [ ]:
print("## VF ##")
print(verbatlas_frames)
#list of frames in the training dataset
l_vf = list(verbatlas_frames.keys())
print(l_vf)
print(len(l_vf))
print("## RL ##")
print(predicate_roles)
p_r = list(predicate_roles.keys())
print(p_r)
print(len(p_r))

We are clearly not using all the 466 [verbatlas](https://verbatlas.org/) frames but less than 3/4 of them: 303. Working with fewer clusters surely increases the overall performances because the system can only focus on a subset of them. In the next code cell I want to check if in the dev-set I do not have other frames.

In [ ]:
dev_sentences, dev_labels = read_dataset("../../data/EN/dev.json")
for k in dev_labels:
    verbatlas_frames.update(dev_labels[k]['predicates'])
    for idx in dev_labels[k]['roles']:
        predicate_roles.update(dev_labels[k]['roles'][idx])

l_vf_dev = list(verbatlas_frames.keys())
print(len(l_vf_dev))

So there are only 4 more frames in the dev_set wrt the train_set, this information is useful for further consideration when I'll deal with the optional part of this homework.

Now that I'm starting to understand the samples, it's clear that our dataset does not need much pre-processing, since we already have words tokens and associated lemmas for each sentence. Some more useful statistics are on how long are the sentences on average, how many predicates they have and how the distribution of pos-tagging tokens correlate with roles and predicates. I'll rapidly compute them in what follows. 

## TODO

## Training

In [16]:
from datasets_srl import SRL_DataModule
from implementation import HParams, SRL_34
from dataclasses import dataclass, asdict
from pprint import pprint
from utils import read_dataset
from mergedeep import merge
hparams = asdict(HParams())
print(hparams)
dev_sentences, dev_labels = read_dataset("../../data/EN/dev.json")
train_sentences, train_labels = read_dataset("../../data/EN/train.json")
task = "34"
sentences = merge(train_labels, train_sentences)
sentences_test = merge(dev_labels, dev_sentences)
data = SRL_DataModule(hparams, task, sentences, sentences_test)
epochs = 10

{'need_train': True, 'batch_size': 3, 'n_cpu': 8, 'language_model_name': 'bert-base-uncased', 'lr': 0.001, 'wd': 1e-05, 'embedding_dim': 768, 'hidden_dim': 512, 'bidirectional': True, 'num_layers': 5, 'dropout': 0.4, 'trainable_embeddings': True, 'role_classes': 27}


In [17]:
data.setup()
batch = iter(data.train_dataloader()).next()
print(batch)
print(batch.keys())

{'input_ids': tensor([[  101,  1996,  2231,  6449,  2008,  5622,  8945,  7507,  2080,  1010,
          5622, 11865, 10932,  1010,  9033, 12022,  7507,  2080,  1010,  5003,
          1062, 20908,  2078,  1998, 19004,  1062,  4048,  5369,  2022, 15389,
          2006,  2382,  2238,  2786,  2044,  2022, 20462,  1997,  4319, 21877,
         21814,  1012,   102, 15389,  3102,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [  101,  1996,  2837,  7297, 16755,  2008,  1996,  2110,  2283,  8627,
          1996,  2865,  2000,  2622,  1037,  3893,  3746,  1997,  2450,  1998,
          1997,  1996,  5020,  3570,  1998,  5368,  1997,  2450,  1998,  2158,
          1999,  1996,  2797,  1998,  2270, 10336,  1012, 

In [18]:
model = SRL_34(hparams=hparams)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
batch["word_id"].dtype

torch.int64

In [20]:
print(model(batch).shape)
print(batch["labels"].shape)

torch.Size([3, 69, 27])
torch.Size([3, 69])


In [24]:
# Define the logger
# https://www.wandb.com/articles/pytorch-lightning-with-weights-biases.
wandb_logger = WandbLogger(project="VAE WM", log_model=True)
wandb_logger.experiment.watch(model, log='all', log_freq=1000)
# Define the trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
metric_to_monitor = 'avg_val_loss_vae'#"loss"
early_stop_callback = EarlyStopping(monitor=metric_to_monitor, min_delta=0.00, patience=15, verbose=True, mode="min")
checkpoint_callback = ModelCheckpoint(
                        save_top_k=1,
                        monitor = metric_to_monitor,
                        mode = "min",
                        dirpath = "../../model",
                        filename ="vae-{epoch:02d}-{avg_val_loss_vae:.4f}",
                        verbose = True
                    )
trainer = pl.Trainer(logger = wandb_logger,
                    max_epochs = epochs, 
                    gpus=1,
                    callbacks=[early_stop_callback, checkpoint_callback])    
# Start the training
trainer.fit(model,data)
# # Log the trained model
# trainer.save_checkpoint("../../model")
wandb.save("../../wandb_logs")



/home/dennis/Applications/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


MisconfigurationException: GPUAccelerator can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].

## TOREMOVE

In [ ]:
from transformers import AutoModel
from transformers import AutoTokenizer

auto_model = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
sequence = "Using a Transformer network is simple"
sequence2 = "we ciao"
# tokens = tokenizer.tokenize(sequence)

# # print(tokens)

# ids = tokenizer.convert_tokens_to_ids(tokens)
# # print(ids)

tokenized_inputs = tokenizer([[sequence, "simple"],[sequence2, "ciao"]],padding=True, return_tensors="pt")  # "pt" -> return PyTorch torch.Tensor objects, rather than a list of tokens

print(tokenized_inputs)
print(tokenized_inputs['input_ids'].shape)
print(tokenized_inputs.word_ids(0))
print(tokenized_inputs.word_ids(1))
# NOTE: in the dataset use those word ids to average and simply filter for example... MAY NOT WORK:...
sequence3 = ["we", "ciao"]
print("second - use this one!!")
tokenized_inputs2 = tokenizer.batch_encode_plus([([sequence.split(), ["simple"]]),(sequence3,["ciao"])], add_special_tokens=True, is_split_into_words=True, padding=True, return_tensors="pt")  # "pt" -> return PyTorch torch.Tensor objects, rather than a list of tokens
print(tokenized_inputs2)
print(tokenized_inputs2['input_ids'].shape)
print(tokenized_inputs2.word_ids(0))
a, b, c = tokenizer.batch_encode_plus([([sequence.split(), ["simple"]]),(sequence3,["ciao"])], add_special_tokens=True, is_split_into_words=True, padding=True, return_tensors="pt")  # "pt" -> return PyTorch torch.Tensor objects, rather than a list of tokens 
print("aaa")
print(b)
print("aaa")
# print(tokenized_inputs2.word_ids(1))
# sequence_a = "This is a short sequence."
# sequence_b = "This is a rather long sequence. It is at least longer than the sequence A."
# print("test")
# padded_sequences = tokenizer([sequence_a, sequence_b], padding=True)
# print(padded_sequences)

transformers_outputs = auto_model(**tokenized_inputs)#['input_ids']
# print(transformers_outputs)
transformers_outputs_sum = torch.stack(transformers_outputs.hidden_states[-4:], dim=0).sum(dim=0)
print(transformers_outputs_sum.shape)
# I should remove 2 sep and 1 cls, 1 additional token -> final size 7



# filter_toke = tokenized_inputs['input_ids'][:, 1:-3, ...]
# print(filter_toke.shape)
# labels  = tokenized_inputs.word_ids()[1:-3]
# samp_size = filter_toke.shape[1]
# M = torch.zeros(max(labels)+1, samp_size)
# M[labels, torch.arange(samp_size)] = 1
# print(M)
# M = torch.nn.functional.normalize(M, p=1, dim=1)
# print(M)
# torch.mm(M, filter_toke[0]).shape

# i want to have an ID to unde
# item["role_id"] = (item["role_labels"] == self.labels_to_id["_"]).long()

In [ ]:
samples = torch.Tensor([[
                     [0.1, 0.1],    #-> group / class 1
                     [0.2, 0.2],    #-> group / class 2
                     [0.4, 0.4],    #-> group / class 2
                     [0.0, 0.0]     #-> group / class 0
              ],
              [
                     [0.1, 0.1],    #-> group / class 1
                     [0.2, 0.2],    #-> group / class 1
                     [0.0, 0.0],    #-> group / class 0
                     [12.0, 12.0]   #-> group / class 0
              ]])

from transformers_embedder.embedder import TransformersEmbedder
labels = torch.LongTensor([[1, 2, 2, 2],[1,2,0,2]])


print(TransformersEmbedder.merge_scatter(samples, labels))
print(TransformersEmbedder.merge_scatter(samples, labels).shape)


In [ ]:
# code taken from Riccardo Orlando transformer embedding https://github.com/Riccorl/transformers-embedder
# it is needed to average the wordpieces after the tokenization to have more reliable embeddig. This is 
# useful because for OOV words (or other languages) we can capture more informations than simply using
# the first token. 
def merge_scatter(embeddings: torch.Tensor, indices: torch.Tensor) -> torch.Tensor:
    """
    Minimal version of ``scatter_mean``, from `pytorch_scatter
    <https://github.com/rusty1s/pytorch_scatter/>`_
    library, that is compatible for ONNX but works only for our case.
    It is used to compute word level embeddings from the transformer output.
    Args:
        embeddings (:obj:`torch.Tensor`):
            The embeddings tensor.
        indices (:obj:`torch.Tensor`):
            The sub-word indices.
    Returns:
        :obj:`torch.Tensor`
    """

    def broadcast(src: torch.Tensor, other: torch.Tensor):
        """
        Broadcast ``src`` to match the shape of ``other``.
        Args:
            src (:obj:`torch.Tensor`):
                The tensor to broadcast.
            other (:obj:`torch.Tensor`):
                The tensor to match the shape of.
        Returns:
            :obj:`torch.Tensor`: The broadcasted tensor.
        """
        for _ in range(src.dim(), other.dim()):
            src = src.unsqueeze(-1)
        src = src.expand_as(other)
        return src

    def scatter_sum(src: torch.Tensor, index: torch.Tensor) -> torch.Tensor:
        """
        Sums the elements in ``src`` that have the same indices as in ``index``.
        Args:
            src (:obj:`torch.Tensor`):
                The tensor to sum.
            index (:obj:`torch.Tensor`):
                The indices to sum.
        Returns:
            :obj:`torch.Tensor`: The summed tensor.
        """
        index = broadcast(index, src)
        size = list(src.size())
        size[1] = index.max() + 1
        print(size)
        print(src.dtype)
        out = torch.zeros(size, dtype=src.dtype, device=src.device)
        return out.scatter_add_(1, index, src)

    # replace padding indices with the maximum value inside the batch
    indices[indices == -1] = torch.max(indices)
    merged = scatter_sum(embeddings, indices)
    ones = torch.ones(
        indices.size(), dtype=embeddings.dtype, device=embeddings.device
    )
    count = scatter_sum(ones, indices)
    count.clamp_(1)
    count = broadcast(count, merged)
    merged.true_divide_(count)
    return merged[:,:-1,:] #added by me to remove a batch!

In [ ]:
torch.tensor([1,2,None],dtype=torch.float)

In [ ]:
pip install transformers-embedder